# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key


In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,westport,41.1415,-73.3579,11.82,94,100,4.63,US,1746853758
1,1,thompson,55.7435,-97.8558,3.90,58,1,2.75,CA,1746853759
2,2,port mathurin,-19.6833,63.4167,26.52,80,27,9.17,MU,1746853761
3,3,waitangi,-43.9535,-176.5597,12.75,70,73,7.65,NZ,1746853762
4,4,manokwari,-0.8667,134.0833,28.43,71,72,2.02,ID,1746853763


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display
import plotly.express as px

# Create the humidity map
fig = px.scatter_geo(
    city_data_df,
    lat="Lat",
    lon="Lng",
    color="Humidity",
    size="Humidity",
    hover_name="City",
    projection="natural earth",
    title="City Humidity Levels",
    color_continuous_scale="Blues"
)

fig.update_layout(
    geo=dict(showland=True, landcolor="lightgray"),
    title_x=0.5
)

# Show the map
fig.show()

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that meet your criteria (example: high humidity & moderate temp)
filtered_cities = city_data_df[
    (city_data_df["Humidity"] <= 65) &
    (city_data_df["Max Temp"] >= 15) &
    (city_data_df["Max Temp"] <= 30)
]

# Drop any rows with null values
filtered_cities = filtered_cities.dropna()

# Display sample data
filtered_cities.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
5,5,al manshah,26.4769,31.8035,28.86,19,1,2.27,EG,1746853764
8,8,blackmans bay,-43.0167,147.3167,18.74,56,72,0.89,AU,1746853767
11,11,alice springs,-23.7000,133.8833,23.75,27,0,4.63,AU,1746853770
28,28,uruacu,-14.5247,-49.1408,22.14,54,28,1.91,BR,1746853790
60,60,forbes,-33.3833,148.0167,21.86,38,0,3.13,AU,1746853826


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create hotel_df
hotel_df = filtered_cities[["City", "Country", "Lat", "Lng", "Humidity"]].copy()

# Add an empty column called "Hotel Name"
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df.head()


,City,Country,Lat,Lng,Humidity,Hotel Name
5,al manshah,EG,26.4769,31.8035,19,
8,blackmans bay,AU,-43.0167,147.3167,56,
11,alice springs,AU,-23.7000,133.8833,27,
28,uruacu,BR,-14.5247,-49.1408,54,
60,forbes,AU,-33.3833,148.0167,38,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
# Test with a known city
test_lat = hotel_df.iloc[0]["Lat"]
test_lon = hotel_df.iloc[0]["Lng"]

params = {
    "categories": "accommodation.hotel",
    "limit": 1,
    "apiKey": geoapify_key,
    "filter": f"circle:{test_lon},{test_lat},10000",
    "bias": f"proximity:{test_lon},{test_lat}"
}

response = requests.get("https://api.geoapify.com/v2/places", params=params)
print(response.url)         # Check the final URL
print(response.status_code) # 200 = OK
print(response.json())      # Inspect the data


https://api.geoapify.com/v2/places?categories=accommodation.hotel&limit=1&apiKey=09280aac4a7e42dea36f657d8027ac24&filter=circle%3A31.8035%2C26.4769%2C10000&bias=proximity%3A31.8035%2C26.4769
200
{'type': 'FeatureCollection', 'features': []}


In [8]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories": "accommodation.hotel",
    "limit": 1,
    "apiKey": geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    lat = row["Lat"]
    lon = row["Lng"]

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lon},{lat},{radius}"
    params["bias"] = f"proximity:{lon},{lat}"


    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    response = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    name_address = response.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
al manshah - nearest hotel: No hotel found
blackmans bay - nearest hotel: Villa Howden
alice springs - nearest hotel: No hotel found
uruacu - nearest hotel: No hotel found
forbes - nearest hotel: No hotel found
el oued - nearest hotel: فندق سوف
codrington - nearest hotel: No hotel found
margaret river - nearest hotel: Margaret River Hotel
ulladulla - nearest hotel: Hotel Marlin
oltu - nearest hotel: No hotel found
saskatoon - nearest hotel: Colonial Square Inn & Suites
viqueque - nearest hotel: Hotel Wailakurini
zhanaozen - nearest hotel: No hotel found
kaifeng - nearest hotel: 喆啡酒店
klyuchi - nearest hotel: No hotel found
laguna - nearest hotel: Holiday Inn Express & Suites
shiyan - nearest hotel: 汉庭酒店
page - nearest hotel: Clarion Inn Page - Lake Powell
altai - nearest hotel: Altai hotel 40000-100000T
zhangye - nearest hotel: Zhangye Hotel 张掖饭店
leninskoye - nearest hotel: No hotel found
arkalyk - nearest hotel: Гостиница "Арсенал"
al qusiyah - nearest hotel: No h

,City,Country,Lat,Lng,Humidity,Hotel Name
5,al manshah,EG,26.4769,31.8035,19,No hotel found
8,blackmans bay,AU,-43.0167,147.3167,56,Villa Howden
11,alice springs,AU,-23.7000,133.8833,27,No hotel found
28,uruacu,BR,-14.5247,-49.1408,54,No hotel found
60,forbes,AU,-33.3833,148.0167,38,No hotel found
...,...,...,...,...,...,...
570,ar rutbah,IQ,33.0381,40.2848,22,No hotel found
574,rice lake,US,45.5061,-91.7382,26,Paul Bunyan Resort
576,faya-largeau,TD,17.9257,19.1043,8,No hotel found
577,zhangzhou,CN,24.5133,117.6556,63,漳州宾馆


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [11]:

import plotly.express as px

# Create a scatter map for hotel locations
fig = px.scatter_geo(
    hotel_df,
    lat="Lat",
    lon="Lng",
    color="Humidity",
    size_max=15,
    hover_name="City",
    hover_data=["Country", "Hotel Name", "Humidity"],
    projection="natural earth",
    title="Recommended Hotel Locations Based on Weather Criteria",
    color_continuous_scale="Blues"
)

fig.update_layout(
    geo=dict(showland=True, landcolor="lightgray"),
    title_x=0.5
)

# Show the map
fig.show()
